# Generating all cones from the maximal ones

The secondary fan on $\operatorname{TGr}_{0}(3,8)$ given to us was a list of rays and maximal cones (up to $\mathfrak{S}_8$--symmetry). In this notebook, we generate the remaining cones. Here is a sketch.

First, a note on the lineality space $L_{\mathbb{R}}$. Each ray $r + L_{\mathbb{R}}$ of  $\operatorname{TGr}^{\circ}(3,8)$ has a unique primitive integral generator in $L_{\mathbb{R}}^{\perp}$. Working with these representatives, we may safely ignore the lineality space. 

Denote the set of $k$-codimensional cones up to $\mathfrak{S}_8$ symmetry by $\mathcal{C}_k$. Given $\mathcal{C}_k$, we want to determine $\mathcal{C}_{k+1}$. First, find all facets of all cones in $\mathcal{C}_k$. This set contains at least one representative from each $\mathfrak{S}_8$-orbit of the $(k+1)$-codimensional cones; we need to find a *unique* representative for each orbit.  For each facet $F$, form the vector $v_F$ by taking the sum of all generating rays, then take the unique primitive generator of this ray. Note that two cones $F,F'$ are equal iff $v_F = v_{F'}$. We use this data to find the unique representatives, and hence form $\mathcal{C}_{k+1}$. 

### Step-by-step guide to generate all cones

For all shell commands, run from the main directory. 

**Generating all rays**  Run ```julia generateAllCones/generateAllRays.jl```. This produces the file ```allRepsByCodim/allRays.dat```, a file whose lines are the rays of $\operatorname{TGr}_{0}(3,8)$ (secondary fan structure), where each ray is a space-separated list of integers.  Note that this contains *all* rays, not just $\mathfrak{S}_8$--orbit representatives. 

**Initiate codimension 0 cones** Run ```julia generateAllCones/codim0_initiate.jl```. This converts the raw $\operatorname{TGr}_{0}(3,8)$ data to the files ```allRepsByCodim/codim0Index.dat``` and ```allRepsByCodim/codim_0.dat```. The file ```codim0Index.dat```  records the maximal cones (up to $\mathfrak{S}_8$--symmetry) by their generating rays. Each line is a space-separated list of integers that represents a single maximal cone, and these numbers correspond to lines in the file ```allRepsByCodim/allRays.dat```. The file ```codim_0.dat``` records representatives of each cone as vectors in their relative interiors. 

**Generate codimension 1 cones** Run ```julia generateAllCones/codim0_to_1.jl```. This produces the files ```allRepsByCodim/codim1Index.dat``` and ```allRepsByCodim/codim_1.dat``` which record the codimension 1 cones (up to $\mathfrak{S}_8$--symmetry) in a way analogous to the codimension 0 cones above. 

**Generate the remaining cones** Run in ```julia``` the files  ```generateAllCones/codim1_to_2.jl```, ```generateAllCones/codim2_to_3.jl```, ```generateAllCones/codim3_to_4.jl```, ```generateAllCones/codim4_to_5.jl```, ```generateAllCones/codim5_to_6.jl```, ```generateAllCones/codim6_to_7.jl```  *in this order*. These produce files like ```allRepsByCodim/codim{j}Index.dat``` and ```allRepsByCodim/codim_{j}.dat``` with ```{j}``` replaced with ```2,3,4,5,6,7```. 

In [1]:
using Oscar
using Combinatorics
pm = Polymake;

  ___   ____   ____    _    ____
 / _ \ / ___| / ___|  / \  |  _ \   |  Combining ANTIC, GAP, Polymake, Singular
| | | |\___ \| |     / _ \ | |_) |  |  Type "?Oscar" for more information
| |_| | ___) | |___ / ___ \|  _ <   |  Manual: https://docs.oscar-system.org
 \___/ |____/ \____/_/   \_\_| \_\  |  Version 1.0.2


In [2]:
cd("..")
currentDir = pwd()
include(joinpath(currentDir, "src/inputData38.jl"));
include(joinpath(currentDir, "src/fileHandling.jl"));
include(joinpath(currentDir, "src/matroidalSubd.jl"));

---
**Function**: ```coneRep2ConeNoLineality(G::Vector{Vector{Int64}},Rs::Matrix{Int64},C::Vector{Tuple{Int64, Int64})```

*Description* The rows of the matrix ```Rs``` are the rays of the fan, up to ```G```-symmetry.  If ```Rs``` has $n$ columns, then ```G``` is subgroup of $\mathfrak{S}_{n}$. The input ```C``` contains the data of a cone as a list of pairs. Each pair contains the data of a ray of the cone, the first entry corresponds to a row of ```Rs```, and the second entry is an entry (index) of ```G```, which says how to permute the entries of the corresponding row. The output is the corresponding polyhedral cone in the orthogonal complement of the lineality space.

---
**Function**: ```cone2RaysFacets(C::Cone)```

*Description* Given a polyhedral cone ```C``` this function returns a pair. The first item is a ```Matrix{Int64}``` whose rows are the rays of ```C```. The second item is a ```Vector{Vector{Int64}}``` which records the facets by their rays, and the rays are recorded by their row index in the ray matrix.

---
**Function**: ```sumRows(M::Matrix{Int64})```

*Description* Returns the sum of the rows of ```M```.

---
**Function**: ```sumRowsPrimitive(M::Matrix{Int64})```

*Description* Returns the unique primitive vector in the direction of the row sum of ```M```.

---
**Function**: ```vector2Primitive(v::Vector{Int64})```

*Description* Returns the unique primitive vector in the direction of ```v```. 

---
**Function**: ```listStrings2File(L::Vector{String}, fileName::String)```

*Description* Returns a file named ```fileName``` whose lines are the entries of ```L```.

---
**Function**: ```orbitVector(G::Vector{Vector{Int64}}, w::Vector{Int64})```

*Description* If ```w``` has $n$ entries, then ```G``` is subgroup of $\mathfrak{S}_{n}$. This function returns ```Set{Vector{Int64}}``` consisting of the orbit of ```w``` where ```G``` acts by permuting the entries of ```w```. 

---
**Function**: ```multisetSupport(v::Vector{Int64})```

*Description* returns the entries of ```v``` sorted. Think of this as the multiset of the elements of ```v```. 

---
**Function**: ```multisetSupp2Vector(ws::Vector{Vector{Int64}})```

*Description* Given a list of vectors ```ws```, this function groups them together by their "multiset" support vector, as in the ```multisetSupport``` function. This is a ```Dict{Vector{Int64}, Vector{Vector{Int64}}}```, the keys are the multiset support vectors, and the values are the vectors that have the given multiset support vector. 

---
**Function**: ```distinctOrbits(G::Vector{Vector{Int64}}, ws::Vector{Vector{Int64}})```

*Description* Here, ```G``` is a subgroup of $\mathfrak{S}_n$ and ```ws``` is a list of vectors of length $n$. This function groups the vectors in ```ws``` into their ```G```-orbits, where ```G``` acts by permuting the entries of the vectors.  The output is a ```Dict{Vector{Int64}, Vector{Vector{Int64}}}```, the keys are the orbit representatives, and the values are the vectors in ```ws``` in the ```G``` orbit of the key. This is made faster by first splitting up ```ws``` using ```multisetSupp2Vector```. 

---
**Function**: ```orbitDict2File(D::Dict, fileName::String)```

*Description* Given a dictionary like the output of ```distinctOrbits```, creates a file called ```fileName``` whose lines are the vectors in the keys of ```D```, each vector is a space-separated list of integers. 

---
**Function**: ```coneRaysUpCodim(higherDim)```

*Description* 

---
**Function**: ```makeOrbitFiles(G, reps, folderName)```

*Description* 


In [ ]:
function coneRep2ConeNoLineality(G,Rs,C)
    dRs = size(Rs);
    GRs = zeros(Int64, length(C), dRs[2]);
    for i in 1:length(C)
        s = C[i]
        GRs[i,:] = Rs[s[1],G[s[2]]]
    end
    return positive_hull(GRs)
end

function cone2RaysFacets(C)
    rs = Matrix{Rational{Int64}}(C.pm_cone.RAYS)
    nrays, dimAmbientSpace = size(rs)
    
    intRs = zeros(Int64, nrays, dimAmbientSpace)
    for i in 1:nrays
        intRs[i,:] = vector2Primitive(rs[i,:])
    end   
    fsRaw = C.pm_cone.RAYS_IN_FACETS
    nfacets, nrays2 = size(fsRaw) 
    fs = [[j for j in 1:nrays if fsRaw[i,j]] for i in 1:nfacets]
    return (intRs, fs)
end

function sumRows(M)
    nrows,ncols = size(M)
    rM = [M[i,:] for i in 1:nrows ]
    return(sum(rM))
end

function sumRowsPrimitive(M)
    v = sumRows(M)
    denom = gcd(v)
    return Vector{Int64}(v/denom)
end

function vector2Primitive(v)
    denom = gcd(v)
    return Vector{Int64}(v/denom)
end

function listStrings2File(L, fileName)
    io = open(fileName,"w")
    newLines = join(L, "\n")
    write(io, newLines)
    close(io)
    return fileName
end

function listVectors2File(L, fileName)
    toListStrings = map(v -> join(map(i->string(i), v), " "), L)
    return listStrings2File(toListStrings, fileName)
end

function orbitVector(G,w)
    return Set{Vector{Int64}}([[w[g[i]] for i in 1:length(w)] for g in G])
end

function multisetSupport(v)
    return sort(v)
end

function multisetSupp2Vector(ws)
    supp2Rep = Dict{Vector{Int64}, Vector{Vector{Int64}}}()
    
    for w in ws
        suppw = multisetSupport(w); 

        if !(suppw in keys(supp2Rep))
            supp2Rep[suppw] = [w]; 
            continue
        end
        if suppw in keys(supp2Rep)
            append!(supp2Rep[suppw], [w]) 
        end
        
    end
    return supp2Rep    
end

function distinctOrbits(G,ws)
    suppDict = multisetSupp2Vector(ws)
    reps = Vector{Vector{Int64}}()
    
    for sw in keys(suppDict)
        
        if length(suppDict[sw]) == 1
            append!(reps, suppDict[sw])
            continue
        end      
        
        rep2Orbit = Dict{Vector{Int64}, Set{Vector{Int64}}}()
        for w in suppDict[sw]             
            if all([!(w in orb) for orb in values(rep2Orbit) ])
                rep2Orbit[w] = orbitVector(G,w)
            end
            
        end
        append!(reps, collect(keys(rep2Orbit)))
    end
    return reps    
end


function orbitDict2File(D, fileName)
    ws = collect(keys(D))
    return listVectors2File(ws, fileName)
end

function coneRaysUpCodim(higherDim)
    interior2Rays = Dict{Vector{Int64}, Matrix{Int64}}()
    for w in keys(higherDim)
        wCone = positive_hull(higherDim[w])
        (wRays,wFacets) = cone2RaysFacets(wCone); 
        for F in wFacets
            raysF = Matrix{Int64}([wRays[i,j] for i ∈ F, j ∈ 1:56 ])
            v = sumRowsPrimitive(raysF); 
            interior2Rays[v] = raysF; 
        end
    end
    return interior2Rays
end

#function makeOrbitFiles(G, reps, folderName)
#    for i in 1:length(reps)
#        orb_i = collect(orbitVector(S8, reps[i]))
#        listVectors2File(orb_i, folderName*"/"*string(i)*".dat")
#    end
#    return folderName
#end